In [81]:

import pandas as pd



In [ ]:
# loop through dictionary matching orpha code (key) to each of its phenotypes (associated values)
def convert_dictionary_df(file_name):
    import pickle
    #load dictionary
    with open(file_name, 'rb') as f:
        dictionary=pickle.load(f)
    master_list=[]
    for orpha, pheno_list in dictionary.items():
        for phenotype in pheno_list:
            list_renew=[]
            list_renew.append(orpha)
            list_renew.append(phenotype)
            master_list.append(list_renew)
    df=pd.DataFrame(data=master_list, columns=['OrphaCode','HPO_phenotype'])
    display(df.head())
    return df
df_orpha_tokens=convert_dictionary_df('processed_data/orpha_ner.pickle')

In [88]:
def HPO_word_count(df,col_name1, col_name2,file_name):
    
    word_count=lambda s:len(s.split())
    df[col_name1]=df[col_name2].apply(word_count)
    display(df.head())
    df.to_csv(f'SQL_scripts/{file_name}_wc.csv')
    return df
df_orpha_tokens_wc=HPO_word_count(df_orpha_tokens,'word_count','HPO_phenotype','orpha_tokens_wc')

,OrphaCode,HPO_phenotype,word_count
0,166024,association multiple epiphyseal dysplasia,4
1,166024,macrocephaly,1
2,166024,dysmorphic facial features,3
3,166024,patients normal stature,3
4,166024,joint swelling,2


In [83]:
#load HPO ontology file to df
def Match_HPO_ID(df, file_name):
    #Import HPO ontology file
    HPO=pd.read_csv('processed_data\HPO.csv')

    # convert to terms to lower case and strip any spaces
    HPO['Term'] = HPO['Term'].str.strip()
    df['HPO_phenotype']=df['HPO_phenotype'].str.strip()
    HPO['Term']=HPO['Term'].str.lower()
    
    #set Terms as index, ready for mapping
    HPO.set_index(['Term'], inplace=True)
    
    
    #Map the hpo phenotye to the id in the HPO ontology file, create new col for id in 
    df['HPO_ID'] = df['HPO_phenotype'].map(HPO['ID'])
    
    #drop and visualise null values
    print('null values before:', df.isna().sum())
    df.dropna(inplace=True)
    print('null values after dropping:', df.isna().sum())
    
    #convert values to integer type
    df['HPO_ID']=df['HPO_ID'].astype('int')
    
    df.to_csv(f'processed_data\{file_name}.csv', index=False)
    
    return df
Match_HPO_ID(df,'extracted_HPO_terms_all')

null values before: OrphaCode           0
HPO_phenotype    1372
HPO_ID           1372
dtype: int64
null values after dropping: OrphaCode        0
HPO_phenotype    0
HPO_ID           0
dtype: int64


,OrphaCode,HPO_phenotype,HPO_ID
0,166024,multiple epiphyseal dysplasia,2654
1,166024,macrocephaly,256
2,166024,joint swelling,1386
3,166024,genu valgum,2857
4,166024,clinodactyly,30084
...,...,...,...
16068,619363,psychosis,709
16069,619233,seizure,1250
16070,619233,anxiety,739
16071,619233,depression,716
